In [2]:
# Celula 0: Instalări (rulează o singură dată)
!pip install torch torchvision
!pip install opencv-python matplotlib numpy
!python -m pip install "git+https://github.com/facebookresearch/detectron2.git"

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\alex\appdata\local\temp\pip-req-build-wrv6rz3f
  Resolved https://github.com/facebookresearch/detectron2.git to commit fd27788985af0f4ca800bca563acdb700bb890e2
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\Alex\AppData\Local\Temp\pip-req-build-wrv6rz3f'


In [3]:
# Celula 1: Importuri
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
import matplotlib.pyplot as plt

IMG_DIR = r"C:\Users\Alex\Desktop\Proiect Lymphonod\Proiect Ultrasound\img" 
ANN_DIR = r"C:\Users\Alex\Desktop\Proiect Lymphonod\Proiect Ultrasound\ann"  

print("Librăriile au fost importate cu succes.")

Librăriile au fost importate cu succes.


In [4]:
# Celula 2: Încărcarea Datelor (Suportă BMP)
from detectron2.structures import BoxMode
import json, os, cv2
from detectron2.data import MetadataCatalog, DatasetCatalog

def get_lymph_node_dicts(img_dir, ann_dir):
    dataset_dicts = []
    
    valid_extensions = ('.jpg', '.png', '.jpeg', '.bmp', '.tif')
    files = [f for f in os.listdir(img_dir) if f.lower().endswith(valid_extensions)]
    
    print(f"Am găsit {len(files)} fișiere de imagine. Încep procesarea...")

    for idx, filename in enumerate(files):
        img_path = os.path.join(img_dir, filename)
        
        json_file = os.path.join(ann_dir, filename + ".json")
        
        if not os.path.exists(json_file):
            json_file = os.path.join(ann_dir, os.path.splitext(filename)[0] + ".json")
            
        if not os.path.exists(json_file):
            # Dacă nici așa nu există, sărim peste
            continue 

        try:
            with open(json_file) as f:
                imgs_anns = json.load(f)
        except:
            print(f"Eroare la citirea JSON: {json_file}")
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue
        height, width = img.shape[:2]
        
        record = {
            "file_name": img_path,
            "image_id": filename,
            "height": height,
            "width": width,
            "annotations": []
        }
      
        objects = []
        if isinstance(imgs_anns, dict) and 'objects' in imgs_anns:
            objects = imgs_anns['objects']
        elif isinstance(imgs_anns, list):
            objects = imgs_anns

        valid_objs = []
        for anno in objects:
            points = []
            if 'points' in anno and 'exterior' in anno['points']:
                points = anno['points']['exterior']
            elif 'geometry' in anno and 'points' in anno['geometry'] and 'exterior' in anno['geometry']['points']:
                points = anno['geometry']['points']['exterior']
            
            if len(points) > 0:
                poly = [float(p) for coord in points for p in coord]
                px = poly[0::2]
                py = poly[1::2]
                
                if len(px) >= 3: # Un poligon trebuie să aibă minim 3 puncte
                    obj = {
                        "bbox": [min(px), min(py), max(px), max(py)],
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "segmentation": [poly],
                        "category_id": 0,
                    }
                    valid_objs.append(obj)
        
        if len(valid_objs) > 0:
            record["annotations"] = valid_objs
            dataset_dicts.append(record)

    print(f" Am încărcat cu succes {len(dataset_dicts)} imagini și adnotări validare.")
    return dataset_dicts

DatasetCatalog.clear()
MetadataCatalog.clear()

DatasetCatalog.register("lymph_train", lambda: get_lymph_node_dicts(IMG_DIR, ANN_DIR))
MetadataCatalog.get("lymph_train").set(thing_classes=["lymph_node"])
print("Dataset înregistrat.")

Dataset înregistrat.


In [5]:
# Celula 3 : Antrenare cu Augmentare 
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import build_detection_train_loader
from detectron2.data import transforms as T
from detectron2.data import DatasetMapper
import os

class LymphTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        augs = [
            T.ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'),
            
            T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
            
            T.RandomRotation(angle=[-45, 45], expand=False),
            
            T.RandomBrightness(0.8, 1.2),
            T.RandomContrast(0.8, 1.2)
        ]
        
        mapper = DatasetMapper(cfg, is_train=True, augmentations=augs)
        return build_detection_train_loader(cfg, mapper=mapper)

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("lymph_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 0 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.MODEL.ROI_MASK_HEAD.POOLER_RESOLUTION = 28 

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025

cfg.SOLVER.MAX_ITER = 600 

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.MODEL.DEVICE = "cpu"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = LymphTrainer(cfg) 
trainer.resume_or_load(resume=False)

print("Start Antrenare Avansată (cu Augmentare și Rezoluție Mare)...")
print("Notă: Pe CPU va dura între 45-90 minute. Nu închide fereastra!")
trainer.train()

print("Antrenare finalizată cu succes!")

[02/12 13:53:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

Start Antrenare Avansată (cu Augmentare și Rezoluție Mare)...
Notă: Pe CPU va dura între 45-90 minute. Nu închide fereastra!
[02/12 13:53:16 d2.engine.train_loop]: Starting training from iteration 0


C:\ProgramData\anaconda3\Lib\site-packages\torch\functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4383.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
W0212 13:53:19.932000 18956 site-packages\torch\fx\_symbolic_trace.py:53] is_fx_tracing will return true for both fx.symbolic_trace and torch.export. Please use is_fx_tracing_symbolic_tracing() for specifically fx.symbolic_trace or torch.compiler.is_compiling() for specifically torch.export/compile.


[02/12 13:55:41 d2.engine.hooks]: Overall training speed: 11 iterations in 0:02:03 (11.2297 s / it)
[02/12 13:55:41 d2.engine.hooks]: Total training time: 0:02:03 (0:00:00 on hooks)
[02/12 13:55:41 d2.utils.events]:  eta: 1:40:15  iter: 13  total_loss: 1.939  loss_cls: 0.7634  loss_box_reg: 0.3755  loss_mask: 0.6891  loss_rpn_cls: 0.0382  loss_rpn_loc: 0.00898    time: 10.5006  last_time: 9.9767  data_time: 0.1299  last_data_time: 0.1163   lr: 5.245e-06  


KeyboardInterrupt: 

In [1]:
# Celula 4 (ULTRA-RAFINATĂ): Contrast Boost + Tight Fit (Eroziune)
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65 
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.ROI_MASK_HEAD.POOLER_RESOLUTION = 28
predictor = DefaultPredictor(cfg)


def enhance_contrast(image):
    """
    Mărește contrastul local (CLAHE) pentru ca marginile să fie BISTURIU.
    """
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

def refine_mask_tight(mask_raw):
    """
    Combină închiderea găurilor cu EROZIUNEA (micșorarea) pentru a strânge conturul.
    """
    kernel_close = np.ones((5,5), np.uint8)
    mask_closed = cv2.morphologyEx(mask_raw, cv2.MORPH_CLOSE, kernel_close)
    
    kernel_erode = np.ones((3,3), np.uint8)
    mask_tight = cv2.erode(mask_closed, kernel_erode, iterations=2)
    
    contours, _ = cv2.findContours(mask_tight, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return mask_raw, None
    
    cnt = contours[0]
    epsilon = 0.003 * cv2.arcLength(cnt, True) 
    approx_curve = cv2.approxPolyDP(cnt, epsilon, True)
    
    return mask_tight, approx_curve

def get_diagnosis(shape_ratio, short_axis_mm):
    score = 0
    if shape_ratio > 0.6: score += 1
    if short_axis_mm > 9.0: score += 1
    
    if score >= 1: return "MALIGN (Suspect)", (255, 0, 0)
    else: return "BENIGN (Normal)", (0, 255, 0)

try: dataset_dicts
except NameError: dataset_dicts = get_lymph_node_dicts(IMG_DIR, ANN_DIR)

d = random.choice(dataset_dicts)
im_original = cv2.imread(d["file_name"])

im_enhanced = enhance_contrast(im_original)

print(f"Analizez imaginea (High Contrast): {os.path.basename(d['file_name'])}")

outputs = predictor(im_enhanced)
instances = outputs["instances"].to("cpu")

img_display = im_original.copy() 

if len(instances) > 0:
    best = instances[0]
    score_ai = best.scores.numpy()[0]
    mask_raw = best.pred_masks[0].numpy().astype('uint8') * 255
    
    mask_final, tight_contour = refine_mask_tight(mask_raw)
    
    if tight_contour is not None:
        if len(tight_contour) >= 5:
            ellipse = cv2.fitEllipse(tight_contour)
            MA, ma = ellipse[1]
            PIXEL_TO_MM = 0.108 
            
            long_axis = max(MA, ma) * PIXEL_TO_MM
            short_axis = min(MA, ma) * PIXEL_TO_MM
            area = cv2.contourArea(tight_contour) * (PIXEL_TO_MM ** 2)
            ratio = short_axis / long_axis
            
            gray = cv2.cvtColor(im_original, cv2.COLOR_BGR2GRAY)
            mask_edge = cv2.Canny(mask_final, 100, 200)
            grad_val = cv2.mean(gray, mask=mask_edge)[0]
            
            diag_text, color_bgr = get_diagnosis(ratio, short_axis)
            color_draw = (color_bgr[2], color_bgr[1], color_bgr[0])
            
            cv2.drawContours(img_display, [tight_contour], -1, color_draw, 2)
            
            infos = [
                f"S/L Ratio:  {ratio:.2f}",
                f"Short Axis: {short_axis:.1f} mm",
                f"Area:       {area:.0f} mm2",
                f"Sharpness:  {grad_val:.1f}",
                f"AI Conf:    {score_ai*100:.0f}%",
                f"RESULT:     {diag_text}"
            ]
            
            y0, dy = 30, 25
            print("\n=== REZULTATE RAFINATE ===")
            for i, line in enumerate(infos):
                print(f"* {line}")
                y = y0 + i * dy
                cv2.putText(img_display, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 4)
                cv2.putText(img_display, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_draw, 2)

    plt.figure(figsize=(15, 8))
    
    plt.subplot(1, 2, 1)
    plt.imshow(im_enhanced[:, :, ::-1])
    plt.title("Ce vede AI-ul (Contrast Boosted)")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(img_display[:, :, ::-1])
    plt.title(f"Rezultat Final (Tight Fit)")
    plt.axis('off')
    
    plt.show()

else:
    print("Nicio detecție. Imaginea e prea dificilă sau contrastul e prea mic.")

NameError: name 'get_lymph_node_dicts' is not defined